<a href="https://colab.research.google.com/github/Ethiongham/Amharic-Hate-Speech/blob/main/Habeshaz_Hatespeech_fine_tuning_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing the transformers package
!pip install transformers
!pip install datasets emoji
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.9 MB/s e

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import argparse


# Define punctuation and diacritics for Amharic text
amharic_punctuations = '''`\r÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ21\n'''
english_punctuations = string.punctuation
punctuations_list = amharic_punctuations + english_punctuations


pos_emoji_list =['😆','🙏','🙂','😛','👍','👌','😄','😸','😃','😅','😂','🤣',"😀","😃","😂","🤣","😊","😍",'😉','😊',
                  '😋','😎','😍','😘','🥰','😗','😙','😚','☺️','🙂','🤗',' 🤩 ','😐',' 💛',' 😇','💛']

neg_emoji_list= ['👿','🖕','😠','😧','😭','😟','😢','😞','😥','🙁','😞','😠','😩','😢','😭','😒','🙄','😣','😥',
                  '😭','😧','😩','😰','😠',' 💔']

def remove_english_character(text):
    text = re.sub(r"[a-zA-Z]+", "", text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    return text

def normalize_amharic(text):
    text = re.sub("[እአኢኤ]", "አ", text)
    text = re.sub("የ", "እ", text)
    text = re.sub("ዉ", "ው", text)
    text = re.sub("ስ", "ሥ", text)
    text = re.sub("ሰ", "ሤ", text)
    text = re.sub("አነ ", "", text)
    return text



def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', str(text))

def clean_function(x):
    x = emoji.demojize(x)
    return x

def text_has_emoji(text):
    for character in text:
        if character in pos_emoji_list:
            return re.sub(character, "ሐምላ", text)
        elif character in neg_emoji_list:
            return re.sub(character, "ሃይዝነት", text)
    return text

def remove_numbers(text):
    return re.sub(r'\b\d+(?:\.\d+)?\s+', '', text)

def amharic_preprocessing(x):
    x = text_has_emoji(x)
    x = remove_repeating_char(x)
    x = ' '.join(word for word in x.split() if word[0] != '#')
    x = remove_punctuations(x)
    x = x.replace('#', '')
    x = re.sub((r"^[\W]*"), "", x)
    x = re.sub((r"\s[\W]\s"), ", ", x)
    x = [i for i in x.split() if len(i) < 16]
    x = " ".join(x)
    x = remove_english_character(x)
    return x



In [ ]:
#importing the datasets package
from datasets import Dataset
import datasets
#import load metric for model evaluation
from datasets import load_metric
#import numpy and pandas for mathematical computation and data manipulation respectively
import numpy as np
import pandas as pd
#import drive package to connect this colab file with the drive where the data will be retrived from
from google.colab import drive
#import the pipeline of transformers
from transformers import pipeline
#import AutoTokenizer for tokenization purposes
from transformers import AutoTokenizer
#import the Trainer API
from transformers import TrainingArguments, Trainer
#import early stopping callback
from transformers import EarlyStoppingCallback, IntervalStrategy
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm
#import train_test_split from sklearn for dividing the dataset into training, testing and validation
from sklearn.model_selection import train_test_split

In [ ]:
#mount google drive to access the dataset directly from the drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- The dataset was obtained with labels and posts in separate files.
- The first step is to merge these files into a single pandas dataframe.
- The second step is label encoding, converting the labels (classes) into numeric format for machine understanding.
- The third step is to divide the dataset into training, validation, and testing sets in a 7:1:2 ratio.
- The final step is to remove unnecessary columns from the main dataset and merge all categories into one main dataset.

In [ ]:
import pandas as pd

# Specify the paths to the renamed files
labels_path = '/Labels.txt'
posts_path = '/Posts.txt'

# Read the Labels file into a Pandas DataFrame
try:
    Labels = pd.read_csv(labels_path, header=None)
    print(f"Successfully read data from '{labels_path}'")
    # Proceed with further processing of Labels DataFrame if needed
except FileNotFoundError:
    print(f"Error: The file '{labels_path}' was not found.")

# Read the Posts file into a Pandas DataFrame
try:
    Posts = pd.read_csv(posts_path, header=None)
    print(f"Successfully read data from '{posts_path}'")
    # Proceed with further processing of Posts DataFrame if needed
except FileNotFoundError:
    print(f"Error: The file '{posts_path}' was not found.")


Successfully read data from '/Labels.txt'
Successfully read data from '/Posts.txt'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#naming the columns
Labels.columns = ["labels"]
Posts.columns = ["post"]
#encoding the classes into numerical data
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1])

In [ ]:
#check the Amharic data
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [ ]:
Posts['post'].iloc[0]

'አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለማገዝ ብሞክር  እርስዎ የመፅሀፍ ቅዱሱ የዳዊት  ኦሮዮንን መንገድ በመምረጥዎ በይፋ ለመፃፍ ተገድጃለሁ  አሁንም መረጃው እንዴት እሱ ጋር ደረሰ የሚለውን ውትወታ ትተው በማንኛውም መንገድ ቀጥተኛ ምላሽ ይስጡኝ   ከመስከረም   ወዲያ አሁን ያለሁ መንግስት ህጋዊ የስራ ዘመኑ ስለሚያበቃ የባለአደራ መንግስት እንዲቋቋም ሆኖም አሁን ያለው የአብይ መንግስት አስፈፃሚ አካል የመንግስትን የእለ ተእለት ተግባራትን እየከወነ ምርጫ እስኪደረግ ለ አመት እ'

In [ ]:
#merge the datasets
Frames = [Labels, Posts]
dataset = pd.concat(Frames, axis=1)


In [ ]:
dataset['post'] = dataset['post'].apply(amharic_preprocessing)

In [ ]:
dataset.head()

,labels,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋምና ክትል እንዲደረግ ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረበት ጥበብና ፍልስፍና ያልከፈተው የእ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ ...


In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import EarlyStoppingCallback


# Read data
# dataset = pd.read_excel("/content/drive/MyDrive/ar_general_sentiment_data.xlsx")
# dataset = dataset[['text','Sentiment']]

# Define pretrained tokenizer and model
model_name = "Davlan/bert-base-multilingual-cased-finetuned-amharic"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# ----- 1. Preprocess data -----#
# Preprocess data
X = list(dataset["post"])
y = list(dataset["labels"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Install the transformers library if not already installed
!pip install transformers

import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset

# Example fine-tuning steps (replace with your actual fine-tuning process)
# Load the pre-trained model and tokenizer
model_name = 'Davlan/bert-base-multilingual-cased-finetuned-amharic'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Example: 2 classes

# Example dataset preparation (replace with your own dataset)
# Assume `train_texts`, `train_labels`, `valid_texts`, `valid_labels` are already defined
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=128)

train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(train_labels))
valid_dataset = TensorDataset(torch.tensor(valid_encodings['input_ids']),
                              torch.tensor(valid_encodings['attention_mask']),
                              torch.tensor(valid_labels))

# Convert datasets to list of dictionaries
train_dataset = [{"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
                 for input_ids, attention_mask, labels in zip(train_encodings['input_ids'],
                                                               train_encodings['attention_mask'],
                                                               train_labels)]
valid_dataset = [{"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
                 for input_ids, attention_mask, labels in zip(valid_encodings['input_ids'],
                                                               valid_encodings['attention_mask'],
                                                               valid_labels)]

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    evaluation_strategy="steps",     # evaluation strategy to adopt during training
    logging_dir="./logs",            # directory for storing logs
    logging_steps=100,               # log every 100 updates
    save_steps=500,                  # save checkpoint every 500 steps
    eval_steps=500,                  # evaluate every 500 steps
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    gradient_accumulation_steps=1,   # gradient accumulation steps
    learning_rate=5e-5,              # learning rate
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    seed=42,                         # random seed
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    metric_for_best_model="accuracy",# use accuracy as the metric to judge the best model
    greater_is_better=True           # higher accuracy means better model
)

# Create Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset           # evaluation dataset
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


TrainOutput(global_step=3, training_loss=0.5857154925664266, metrics={'train_runtime': 13.1414, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.228, 'total_flos': 24666661440.0, 'train_loss': 0.5857154925664266, 'epoch': 3.0})

In [ ]:
!pip install transformers


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'Davlan/bert-base-multilingual-cased-finetuned-amharic'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Example: 2 classes


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip show accelerate


Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
import os
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding

# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Step 1: Define your dataset (Example dataset)
train_texts = ["Example sentence 1", "Example sentence 2"]
train_labels = [0, 1]  # Example labels
valid_texts = ["Example validation sentence 1", "Example validation sentence 2"]
valid_labels = [0, 1]  # Example validation labels

# Step 2: Tokenize the dataset
model_name = 'Davlan/bert-base-multilingual-cased-finetuned-amharic'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels based on your task

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=128)

train_dataset = CustomDataset(train_encodings, train_labels)
valid_dataset = CustomDataset(valid_encodings, valid_labels)

# Step 3: Define Training Arguments and Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    eval_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# Use DataCollatorWithPadding for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator
)

# Step 4: Train the model
trainer.train()

# Step 5: Evaluate the Model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Step 6: Save the Model and Tokenizer
# Define the path where you want to save the model
model_path = "Habeshaztext hatespeech_Model"

# Check if the directory exists, if not, create it
if not os.path.exists(model_path):
    os.makedirs(model_path)

# Save the model and tokenizer
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Evaluation results: {'eval_loss': 0.7047601938247681, 'eval_runtime': 0.1341, 'eval_samples_per_second': 14.918, 'eval_steps_per_second': 7.459, 'epoch': 3.0}


('Habeshaztext hatespeech_Model/tokenizer_config.json',
 'Habeshaztext hatespeech_Model/special_tokens_map.json',
 'Habeshaztext hatespeech_Model/vocab.txt',
 'Habeshaztext hatespeech_Model/added_tokens.json')

In [ ]:
hatespeech_Model = model_path = '/content/drive/MyDrive/Habeshaztext'
pretrained_model = "Davlan/bert-base-multilingual-cased-finetuned-amharic"

In [ ]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model)
model = BertForSequenceClassification.from_pretrained(hatespeech_Model, num_labels=2)

In [ ]:
from transformers import BertTokenizer

# Assuming `tokenizer` is already initialized
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')  # Replace with your tokenizer

# Example text
text = "Your test text goes here."

# Tokenize the text
X_test_tokenized = tokenizer(text, padding=True, truncation=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import TensorDataset

# Convert tokenized inputs to tensors
input_ids = torch.tensor(X_test_tokenized['input_ids']).unsqueeze(0)  # Assuming a single example
attention_mask = torch.tensor(X_test_tokenized['attention_mask']).unsqueeze(0)  # Assuming a single example

# Create the dataset
test_dataset = TensorDataset(input_ids, attention_mask)


In [ ]:
text = [
    'እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደረግ በመግለጫው ጠይቀዋል',
    'አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለማገዝ ብሞክር  እርስዎ የመፅሀፍ ቅዱሱ የዳዊት  ኦሮዮንን መንገድ በመምረጥዎ በይፋ ለመፃፍ ተገድጃለሁ  አሁንም መረጃው እንዴት እሱ ጋር ደረሰ የሚለውን ውትወታ ትተው በማንኛውም መንገድ ቀጥተኛ ምላሽ ይስጡኝ',
    'ከመስከረም   ወዲያ አሁን ያለሁ መንግስት ህጋዊ የስራ ዘመኑ ስለሚያበቃ የባለአደራ መንግስት እንዲቋቋም ሆኖም አሁን ያለው የአብይ መንግስት አስፈፃሚ አካል የመንግስትን የእለ ተእለት ተግባራትን እየከወነ ምርጫ እስኪደረግ ለ አመት እ'
]

# Tokenize the text
X_test_tokenized = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")


In [ ]:
model_path = "/content/drive/MyDrive/Habeshaztext hatespeech_Model"

# Load trained model
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)


In [ ]:
text = [
    'እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደረግ በመግለጫው ጠይቀዋል',
    'አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለማገዝ ብሞክር  እርስዎ የመፅሀፍ ቅዱሱ የዳዊት  ኦሮዮንን መንገድ በመምረጥዎ በይፋ ለመፃፍ ተገድጃለሁ  አሁንም መረጃው እንዴት እሱ ጋር ደረሰ የሚለውን ውትወታ ትተው በማንኛውም መንገድ ቀጥተኛ ምላሽ ይስጡኝ',
    'ከመስከረም   ወዲያ አሁን ያለሁ መንግስት ህጋዊ የስራ ዘመኑ ስለሚያበቃ የባለአደራ መንግስት እንዲቋቋም ሆኖም አሁን ያለው የአብይ መንግስት አስፈፃሚ አካል የመንግስትን የእለ ተእለት ተግባራትን እየከወነ ምርጫ እስኪደረግ ለ አመት እ'
]

# Tokenize the text
X_test_tokenized = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")


In [ ]:
from transformers import BertForSequenceClassification

model_path = "/content/drive/MyDrive/Habeshaztext hatespeech_Model"

# Load trained model
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)


In [ ]:
# Assuming you have already tokenized the test data as X_test_tokenized

# Load trained model
from transformers import BertForSequenceClassification

model_path = "/content/drive/MyDrive/Habeshaztext hatespeech_Model"

model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Perform inference
import torch

# Ensure the model is in evaluation mode
model.eval()

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: value.to(device) for key, value in X_test_tokenized.items()}

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Assuming outputs is a tuple (logits, ...)
logits = outputs.logits

# Post-process logits if needed
predictions = torch.argmax(logits, dim=-1)

# Convert predictions to numpy array
y_pred = predictions.cpu().numpy()

# Now y_pred contains the predicted labels
print(y_pred)


[1 1 1]


1 - Hatespeech
0 - Normal Speech

In [ ]:
y_pred

array([1, 1, 1])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
y_true = [0, 1, 1]  # Example true labels (0 and 1 correspond to different classes)


In [ ]:
accuracy = accuracy_score(y_true, y_pred)


In [ ]:
precision = precision_score(y_true, y_pred)


In [ ]:
recall = recall_score(y_true, y_pred)


In [ ]:
f1 = f1_score(y_true, y_pred)


In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 1.0
F1-score: 0.8


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Example predicted and true labels
y_pred = [1, 1, 1]  # Example predicted labels
y_true = [0, 1, 1]  # Example true labels

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 1.0
F1-score: 0.8
